In [1]:
import os
import numpy as np
import math

def _read_txt_(url):
    file = open(url, 'r', encoding='utf-8')
    seg_list = []
    lines = file.readlines()
    for i in range(len(lines)):
        seg_list.append(lines[i].rstrip("\n").split('\t'))
        pass
    file.close()
    return seg_list

# format the row data
# information of ID corresponds to index of list
# therefore we only need head, words, pos
def _format_(row_data):
    train_head, train_words, train_pos = [], [], []
    tmp_head, tmp_words, tmp_pos = [], [], []
    for seq in row_data:
        if len(seq) == 1:
            # save last one sentence
            train_head.append(tmp_head)
            train_words.append(tmp_words)
            train_pos.append(tmp_pos)
            # clean
            tmp_head, tmp_words, tmp_pos = [], [], []
            continue
        tmp_head.append(seq[-2])
        if seq[1].isdigit():
            tmp_words.append("NUM")
        elif "." in seq[1] and len(seq[1]) > 1:
            if "." == seq[1][-1]:
                tmp_words.append("ABBRE")
            else:
                tmp_words.append("DEC")
        elif "," in seq[1] and len(seq[1]) > 1:
            if seq[1].split(",")[0].isdigit():
                tmp_words.append("NUM")
        elif seq[1].isupper():
            tmp_words.append("SUPPER")
        else:
            tmp_words.append(seq[1])
        tmp_pos.append(seq[3])
    return train_head, train_words, train_pos

train_row = _read_txt_("mstparser-en-train.dep")
train_head, train_words, train_pos = _format_(train_row)

test_row = _read_txt_("mstparser-en-test.dep")
test_head, test_words, test_pos = _format_(test_row)

In [2]:
def _features_(stack, queue):
    features = []
    # combine features
    # word -2 -> word -1
    features.append(stack[-2][1] + " " + stack[-1][1])
    # word -2 -> pos -1
    features.append(stack[-2][1] + " " + stack[-1][2])
    # pos -2 -> word -1
    features.append(stack[-2][2] + " " + stack[-1][1])
    # pos -2 -> pos -1
    features.append(stack[-2][2] + " " + stack[-1][2])
    # word -1 -> word 0
    features.append(stack[-1][1] + " " + queue[0][1])
    # word -1 -> pos 0
    features.append(stack[-1][1] + " " + queue[0][2])
    # pos -1 -> word 0
    features.append(stack[-1][2] + " " + queue[0][1])
    # pos -1 -> pos 0
    features.append(stack[-1][2] + " " + queue[0][2])
    return features

In [3]:
def _computation_(w, features,split="train"):
    if split == "train":
        # check existing of w 
        for feature in features:
            if feature not in w:
                w[feature] = 1
    score = 0
    for feature in features:
        if feature in w:
            value = w[feature]
            if value > 0:
                value = math.log10(value + 0.1)
            elif value < 0:
                value = -math.log10(abs(value) + 0.1)
            score += value
        else:
            # penalty for UNK
            score += math.log10(abs(3) + 0.1)
    return score
def _update_(w_ans,features, w_corr=None, mode="award"):
    # predict successfully, mode = award, + 1
    # predict unsuccessfully, mode = penalty, -2 in answer, +2 in correct
    if mode == "award":
        for feature in features:
            w_ans[feature] += 1
    else:
        for feature in features:
            w_ans[feature] += -10
            w_corr[feature] += 10
#     if mode != "award":
#         or feature in features:
#             w_ans[feature] += -1
#             w_corr[feature] += 1

In [4]:
w_s, w_r, w_l = {}, {}, {}
SHIFT = "SHIFT"
LEFT = "LEFT"
RIGHT = "RIGHT"
# train
for i in range(len(train_head)):
    real_heads, words, pos = train_head[i], train_words[i], train_pos[i]
    # init heads_ref & children
    heads_ref = {0:-1}
    children = {}
    for num in range(len(real_heads) + 1):
        if num not in children:
            children[num] = 0
    for idx, head in enumerate(real_heads):
        heads_ref[int(idx+1)] = int(head)
        children[int(head)] = children[int(head)] + 1
    # init stack
    stack = [[0, 'root', "ROOT"], [1, words[0], pos[0]]]
    heads = {0:-1}
    # combine queue
    queue = []
    for j in range(1,len(words)):
        queue.append([int(j+1), words[j], pos[j]])
    queue.append([len(words)+1, "</s>","</s>"])
    # begin train
    while len(queue) > 1 or len(stack) > 1:
        ans, corr = "", ""
        features = _features_(stack, queue)
        s_s = _computation_(w_s, features)
        s_r = _computation_(w_r, features)
        s_l = _computation_(w_l, features)
        # answer from perceptron
        if s_s >= s_l and s_s >= s_r and len(queue)>0:
            ans = SHIFT
        elif s_l >= s_r:
            ans = LEFT
        else:
            ans = RIGHT
        # correct answer
        if (heads_ref[stack[-1][0]]== stack[-2][0]) and children[stack[-1][0]] == 0:
            corr = RIGHT
        elif heads_ref[stack[-2][0]]== stack[-1][0] and children[stack[-2][0]] == 0:
            corr = LEFT
        else:
            corr = SHIFT
        if ans == corr:
            # award
            if ans == SHIFT:
                _update_(w_s, features)
            elif ans == RIGHT:
                _update_(w_r, features)
            else:
                _update_(w_l, features)
        else:
            # penalty
            if ans == SHIFT:
                if corr == RIGHT:
                    _update_(w_s, features,w_corr=w_r, mode="penalty")
                else:
                    _update_(w_s, features,w_corr=w_l, mode="penalty")
            elif ans == RIGHT:
                if corr == LEFT:
                    _update_(w_r, features,w_corr=w_l, mode="penalty")
                else:
                    _update_(w_r, features,w_corr=w_s, mode="penalty")
            else:
                if corr == SHIFT:
                    _update_(w_l, features,w_corr=w_s, mode="penalty")
                else:
                    _update_(w_l, features,w_corr=w_r, mode="penalty")
        if corr == SHIFT:
            stack.append(queue[0])
            queue.pop(0)
        elif corr == LEFT:
            heads[stack[-2][0]] = stack[-1][0]
            children[stack[-1][0]] = children[stack[-1][0]] - 1
            stack.pop(-2)
        else:
            heads[stack[-1][0]] = stack[-2][0]
            children[stack[-2][0]] = children[stack[-2][0]] - 1
            stack.pop(-1)

In [5]:
# test
numerator = 0
denominator = 0
for i in range(len(test_head)):
    real_heads, words, pos = test_head[i], test_words[i], test_pos[i]
    heads_ref = {0:-1}
    for idx, head in enumerate(real_heads):
        heads_ref[int(idx+1)] = int(head)
    # init stack
    stack = [[0, 'root', "ROOT"], [1, words[0], pos[0]]]
    heads = {0:-1}
    # combine queue
    queue = []
    for j in range(1,len(words)):
        queue.append([int(j+1), words[j], pos[j]])
    queue.append([len(words)+1, "</s>","</s>"])
    # begin compute
    while len(queue) > 1 or len(stack) > 1:
        features = _features_(stack, queue)
        s_s = _computation_(w_s, features)
        s_r = _computation_(w_r, features)
        s_l = _computation_(w_l, features)
        #  predict
        if s_s >= s_l and s_s >= s_r and len(queue)>1:
            stack.append(queue[0])
            queue.pop(0)
        elif s_l >= s_r:
            heads[stack[-2][0]] = stack[-1][0]
            stack.pop(-2)
        else:
            heads[stack[-1][0]] = stack[-2][0]
            stack.pop(-1)
            
        if len(stack) == 1 and len(queue) > 1:
            stack.append(queue[0])
            queue.pop(0)
    for j in range(len(heads_ref)):
        if j not in heads or j not in heads_ref:
            continue
        if heads[j] == heads_ref[j]:
            numerator += 1
    denominator += len(real_heads)

In [6]:
print("acc: " + str(round(numerator/denominator,3)*100) + "%")

acc: 51.1%
